## neo4j 图数据库操作

### 导入相关依赖

```bash
uv add py2neo # neo4j 的 python 客户端
uv add tqdm # 进度条显示库
uv add prettyprinter # 增强的数据格式化输出库
```

In [1]:
from py2neo import Graph, Node, Relationship

In [2]:
import json
from tqdm import tqdm
from pprint import pprint

### 连接到 Neo4j 数据库

In [3]:
graph = Graph("bolt://localhost:7687", user="neo4j", password="neo4j123", name="neo4j")

### 解析数据

#### 公司事件数据解析

In [4]:
event_type_file = './data/iree.json'
event_type_data =[]
with open(event_type_file, 'r', encoding='utf-8') as file:
    for line in file.readlines():
        event_type_data.append(json.loads(line.strip()))

pprint(event_type_data[2], sort_dicts=True)

{'content': '据美国《纽约时报》29日报道，全球航空业放缓正对波音公司造成巨大打击。波音公司在29日表示，将裁员约1.6万人，此前该公司报告称，今年前三个月收入锐减26%。波音首席执行官戴维·卡尔霍恩在给员工的一份报告中说，“全球疫情大流行改变了人们的生活和工作方式。它正在改变我们的行业，我们面临着完全意想不到的挑战。”一家行业商会称，全世界的航空公司都在努力生存，到今年年底，航空业亏损总额预计将超过3000亿美元。因此，许多航空公司正在推迟购买、交付和维修。波音公司表示，正在放缓飞机的生产，包括陷入困境的737 '
            'Max、787、777和777x机型。该公司还在探索从美国联邦政府或金融市场筹集更多资金的方法。此次裁员约占波音员工总数的10%，对于那些受经济衰退影响最大的部门——商用飞机和服务行业——的员工来说，裁员幅度将更大，这些部门的裁员幅度约为15%。卡尔霍恩在报告中对雇员表示：“我知道，在一个已经充满挑战的时期，这个消息是一个打击。我很遗憾这会对你们当中的许多人产生影响。我衷心希望还有其他方法。”波音公司报告第一季度净亏损6.41亿美元，而去年同期则盈利21亿美元。该公司此前曾表示，预计航空旅行在三年内不会恢复到疫情暴发前的水平，而且很可能还需要几年时间，旅行才能恢复到以前的长期增长率。波音的商用飞机业务在第一季度受到737 '
            'Max机型和新冠病毒大流行的打击尤其严重，该业务的收入比去年同期下降近50%，降至62亿美元。公司总收入下降至169亿美元。该公司在1月至3月仅收到49笔新订单，有196笔订单被取消。西南航空(Southwest '
            'Airlines)在28日表示，该公司一直在就减少今年将接收的737 '
            'Max飞机数量进行谈判。西南航空表示，到2021年底，该公司最多将接收48架Max喷气式飞机，而不是此前预期的107架。波音表示，希望通过包括买断和提前退休的自愿方式来实现裁员目标。波音公司上周告诉员工，接受买断的员工依照在波音工作时长，每年将获得3个月的医疗保险和1周的工资，最高可达26年。员工必须在周一之前表明他们对买断的兴趣。如果获得批准，他们将在6月初离职。报道称，波音的裁员将可能不成比例地集中在波音位于华盛顿州和南卡罗来

#### 投资数据

In [5]:
invest_file = './data/invest-on-invent-KG.json'
with open(invest_file, 'r', encoding='utf-8') as file:
    invest_data = json.load(file)

##### 投资者数据

In [6]:
invest_data['@graph'][0]

{'@id': '0',
 '@type': 'investor',
 'name': '瑞华林投资',
 'relationship': {'investCompany': [{'@id': '5617',
    '@type': 'company',
    'round': '新三板定增',
    'date': '2016-03-04'}]}}

##### 公司信息

In [7]:
invest_data['@graph'][5323]

{'@id': '5323',
 '@type': 'company',
 'name': '福建博思软件股份有限公司',
 'alterNames': ['博思软件',
  '福建博思软件',
  '博思软件公司',
  '福建省博思软件',
  '福建博思软件公司',
  '福建省博思软件公司'],
 'establishDate': '2001-09-05',
 'address': '闽侯县上街镇高新大道5号',
 'relationship': {'applyPatent': [{'@id': '17314', '@type': 'patent'},
   {'@id': '17315', '@type': 'patent'},
   {'@id': '17316', '@type': 'patent'},
   {'@id': '17317', '@type': 'patent'},
   {'@id': '17318', '@type': 'patent'},
   {'@id': '17319', '@type': 'patent'},
   {'@id': '17320', '@type': 'patent'}]}}

##### 专利信息

In [8]:
invest_data['@graph'][141764]

{'@id': '141764',
 '@type': 'patent',
 'name': '一种基于DLL的软件架构',
 'patentType': '发明公布'}

### 数据处理

- 投资数据中：投资者和投资公司、公司和专利 id 的对应关系
- 投资数据的公司名称和事件数据名称的对应关系

In [9]:
obj_by_id = {} # key: id, value: object

for ele in invest_data['@graph']:
    obj_by_id[ele['@id']] = ele

#### 提取事件数据中的公司信息

{'content': '...',
 'label': [{'arguments': [{'时间': '29日'}, {'主体': '波音公司'}], 'event_type': '裁员'},
           {'arguments': [{'主体': '波音'}, {'数值': ['比去年同期下降近50%']}],
            'event_type': '主营业务收入减少'}],
 'title': '疫情冲击下波音将裁员10％，员工可选买断工龄或提前退休'}

如上所示，content.label[i].arguments[j].主体 的值为公司名称，需要提取出来

In [10]:
ignore_names = ['电器', '600万股', '公司', '135万元', '2018年', '新产业'] # 脏数据
event_type_company_names = {}
for data in event_type_data:
    for label in data['label']:
        if 'arguments' in label:
            for arg in label['arguments']:
                for key, value in arg.items():
                    if key == '主体':
                        if value in ignore_names:
                            continue
                        event_type_company_names[value] = 1

list(event_type_company_names.keys())[:10]

['小米集团',
 '小米',
 '格力电器',
 '波音公司',
 '波音',
 'GoPro',
 '得邦照明股份有限公司',
 '得邦照明',
 '横店集团得邦工程塑料有限公司',
 '汉邦高科']

#### 提取投资关系数据中的公司名称

{'@id': '6382',
 '@type': 'company',
 'name': '广州中设机器人智能装备股份有限公司',
 'alterNames': ['中设机器人智能装备',
  '中设机器人智能装备公司',
  '广州中设机器人智能装备',
  '广州市中设机器人智能装备',
  '广州中设机器人智能装备公司',
  '广州市中设机器人智能装备公司'],
 'establishDate': '2008-07-23',
 'address': '广州市黄埔区云埔工业区方达路6号101房',
 'relationship': {'applyPatent': []}}

 如上所示，当 @type 为 company 时，表示这个公司数据，需要提取 name 字段

In [11]:
invest_company_names = []
for ele in invest_data['@graph']:
    if ele['@type'] == "company":
        invest_company_names.append(ele['name'])

pprint(invest_company_names[:10])

['广州博鳌纵横网络科技有限公司',
 '福建博思软件股份有限公司',
 '深圳优地科技有限公司',
 '杭州群核信息技术有限公司',
 '昆山希盟自动化科技有限公司',
 '北京小爱智能科技有限公司',
 '奇安信科技集团股份有限公司',
 '北京大米科技有限公司',
 '北京诸葛找房信息技术有限公司',
 '苏州奥易克斯汽车电子有限公司']


#### 建立投资关系数据和事件数据中的公司关系

因为投资关系数据中的公司名称是完整的，所以以投资关系数据中的公司名称为基准，进行字符串包含判断，如果包含，则建立关系。

In [12]:
event_type_company_dict = {} # key: 事件关系中的公司名称；value: 投资数据中的公司名称列表
for company in event_type_company_names.keys():
    for iv_name in invest_company_names:
        if company in iv_name:
            event_type_company_dict.setdefault(company, []).append(iv_name) # 建立一对多关系

pprint(event_type_company_dict['小米'])
pprint(event_type_company_dict['瑞幸咖啡'])

['小米科技有限责任公司']
['瑞幸咖啡(北京)有限公司']


### 构建图数据库

#### 构建节点和边

- 公司和事件
- 公司和投资者
- 公司和专利

##### 公司和事件

```json
{
    "content": "...",
    "label": [
        {
            "arguments": [
                {
                    "时间": "29日"
                },
                {
                    "主体": "波音公司"
                }
            ],
            "event_type": "裁员"
        },
        {
            "arguments": [
                {
                    "主体": "波音"
                },
                {
                    "数值": [
                        "比去年同期下降近50%"
                    ]
                }
            ],
            "event_type": "主营业务收入减少"
        }
    ],
    "title": "疫情冲击下波音将裁员10％，员工可选买断工龄或提前退休"
}
```

In [13]:
for data in tqdm(event_type_data):
    title = data['title']
    content = data['content']

    for label in data['label']:
        event_type = label['event_type']
        event_type_node = Node("EventType", name=event_type)  # 建立事件类型节点
        graph.merge(event_type_node, "EventType", "name") # 添加到图数据库中，name 是主键属性的名称

        if 'arguments' in label:
            company_name = ''
            tmp_args = {}
            for arg in label['arguments']:
                for key, value in arg.items():
                    if key == '主体':
                        company_name = value
                    else:
                        tmp_args[key] = value
                if company_name in event_type_company_dict:
                    for cname in event_type_company_dict[company_name]:
                        company_node = Node("Company", name=cname) # 建立公司节点
                        graph.merge(company_node, "Company", "name")

                        relationship = Relationship(company_node, "HAPPEN", event_type_node) # 建立公司与事件的关系
                        for key ,value in tmp_args.items():
                            relationship[key] = value
                        relationship['title'] = title
                        relationship['content'] = content
                        graph.create(relationship) # 存储到图数据库中


100%|██████████| 7058/7058 [01:15<00:00, 93.62it/s] 


##### 公司与专利

```json
{
    "@id": "5323",
    "@type": "company",
    "name": "福建博思软件股份有限公司",
    "alterNames": ["博思软件"],
    "establishDate": "2001-09-05",
    "address": "闽侯县上街镇高新大道5号",
    "relationship": {
        "applyPatent": [
            {
                "@id": "17314",
                "@type": "patent"
            }
        ]
    }
}
```

In [14]:
# 在数据导入前创建索引
def create_indexes(graph):
    """创建索引提升查询性能"""
    indexes = [
        "CREATE INDEX company_name IF NOT EXISTS FOR (c:Company) ON (c.name)",
        "CREATE INDEX patent_name IF NOT EXISTS FOR (p:Patent) ON (p.name)",
        "CREATE INDEX investor_name IF NOT EXISTS FOR (i:Investor) ON (i.name)",
        "CREATE INDEX event_type_name IF NOT EXISTS FOR (e:EventType) ON (e.name)"
    ]
    
    for index_cypher in indexes:
        try:
            graph.run(index_cypher)
            print(f"索引创建成功: {index_cypher}")
        except Exception as e:
            print(f"索引创建失败: {e}")

# 在导入数据前执行
create_indexes(graph)

# 2. 批量处理公司和专利数据
def company_patent_import():
    print("开始批量导入公司和专利数据...")
    
    # 一次性准备所有数据
    companies = []
    relationships = []
    
    for ele in tqdm(invest_data['@graph'], desc="预处理公司专利数据"):
        if ele['@type'] == "company":
            companies.append({
                'name': ele['name'],
                'establishDate': ele['establishDate'],
                'address': ele['address']
            })
            
            for patent in ele['relationship']['applyPatent']:
                patent_id = patent['@id']
                if patent_id not in obj_by_id:
                    continue
                patent_name = obj_by_id[patent_id]['name']
                relationships.append({
                    'company': ele['name'],
                    'patent': patent_name
                })
    
    # 批量创建公司节点
    batch_size = 1000
    for i in tqdm(range(0, len(companies), batch_size), desc="创建公司节点"):
        batch = companies[i:i + batch_size]
        cypher = """
        UNWIND $companies as company
        MERGE (c:Company {name: company.name})
        SET c.establishDate = company.establishDate, c.address = company.address
        """
        graph.run(cypher, companies=batch)
    
    # 批量创建专利和关系
    for i in tqdm(range(0, len(relationships), batch_size), desc="创建专利和关系"):
        batch = relationships[i:i + batch_size]
        cypher = """
        UNWIND $relationships as rel
        MERGE (p:Patent {name: rel.patent})
        WITH p, rel
        MATCH (c:Company {name: rel.company})
        MERGE (c)-[:HAVE]->(p)
        """
        graph.run(cypher, relationships=batch)

# 执行导入
company_patent_import()

索引创建成功: CREATE INDEX company_name IF NOT EXISTS FOR (c:Company) ON (c.name)
索引创建成功: CREATE INDEX patent_name IF NOT EXISTS FOR (p:Patent) ON (p.name)
索引创建成功: CREATE INDEX investor_name IF NOT EXISTS FOR (i:Investor) ON (i.name)
索引创建成功: CREATE INDEX event_type_name IF NOT EXISTS FOR (e:EventType) ON (e.name)
开始批量导入公司和专利数据...


创建专利和关系: 100%|██████████| 183/183 [00:03<00:00, 48.60it/s]


##### 公司与投资者


```json
{
    "@id": "0",
    "@type": "investor",
    "name": "瑞华林投资",
    "relationship": {
        "investCompany": [
            {
                "@id": "5617",
                "@type": "company",
                "round": "新三板定增",
                "date": "2016-03-04"
            }
        ]
    }
}
```

In [15]:
def company_investor_import():
    investors = []
    companies = {} # 用字典去重
    relationships = []

    # 提前准备好数据
    for ele in tqdm(invest_data['@graph'], desc="预处理公司专利数据"):
        if ele['@type'] == "investor":
            investors.append({
                'name': ele['name']
            })

            invest_company = ele['relationship']['investCompany']
            for invest in invest_company:
                invest_id = invest['@id']
                if invest_id not in obj_by_id:
                    continue
                company_obj = obj_by_id[invest_id]
                company_name = company_obj['name']
                companies[company_name] = {
                    'name': company_name,
                    'establishDate': company_obj['establishDate'],
                    'address': company_obj['address']
                }
                relationships.append({
                    'investor_name': ele['name'],
                    'company_name': company_name,
                    'date': invest['date'],
                    'round': invest['round']
                })
    
    # 批量创建投资者节点
    batch_size = 1000
    for i in tqdm(range(0, len(investors), batch_size), desc="创建投资者节点"):
        batch = investors[i:i + batch_size]
        cypher = """
        UNWIND $investors as investor
        MERGE (i:Investor {name: investor.name})
        """
        graph.run(cypher, investors=batch)
    
    # 批量创建公司
    companies_list = list(companies.values())
    for i in tqdm(range(0, len(companies_list), batch_size), desc="创建公司节点"):
        batch = companies_list[i:i + batch_size]
        cypher = """
        UNWIND $companies as company
        MERGE (c:Company {name: company.name})
        SET c.establishDate = company.establishDate, c.address = company.address
        """
        graph.run(cypher, companies=batch)

    # 批量创建投资关系
    for i in tqdm(range(0, len(relationships), batch_size), desc="创建投资关系"):
        batch = relationships[i:i + batch_size]
        cypher = """
        UNWIND $relationships as rel
        MATCH (i:Investor {name: rel.investor_name})
        MATCH (c:Company {name: rel.company_name})
        MERGE (i)-[r:INVEST]->(c)
        SET r.date = rel.date, r.round = rel.round
        """
        graph.run(cypher, relationships=batch)

company_investor_import()

创建投资关系: 100%|██████████| 12/12 [00:00<00:00, 35.13it/s]


### Cypher 语句使用

#### MATCH 查询

In [16]:
query = """
MATCH (company)-[r:HAPPEN]->(e:EventType {name: "业绩承诺未达标"})
RETURN company.name as name, r.title as title
"""

results = graph.run(query)
for record in results:
    print(dict(record))

{'name': '北京真视通科技股份有限公司', 'title': ''}
{'name': '上海中锂实业有限公司', 'title': ''}
{'name': '浙江南都电源动力股份有限公司', 'title': ''}
{'name': '上海科匠信息科技有限公司', 'title': ''}
{'name': '安徽晶奇网络科技股份有限公司', 'title': ''}
{'name': '阳光城集团股份有限公司', 'title': ''}
{'name': '浙江聚力文化发展股份有限公司', 'title': ''}
{'name': '青岛万达影视投资有限公司', 'title': '商誉暴雷，业绩亏损，万达电影缘何还能逆势扩张'}
{'name': '浙江田中精机股份有限公司', 'title': '0.3折“卖子”给大股东 田中精机被疑利益输送'}
{'name': '高斯贝尔数码科技股份有限公司', 'title': '高斯贝尔持续经营能力遭深交所质疑 子公司业绩承诺期业绩均未'}
{'name': '中文在线数字出版集团股份有限公司', 'title': '市场动态 监管动态 | 收购子公司未达业绩承诺，中文在线收北京证监局警示函'}
{'name': '广东东方精工科技股份有限公司', 'title': '卷进东方精工业绩纠纷，宁德时代称对方的披露严重失实'}
{'name': '鄂州二医院有限公司', 'title': '上市公司医院投资项目暴雷 医院业绩暴跌'}
{'name': '博瑞生物医药(苏州)股份有限公司', 'title': '博瑞生物签订4项对赌协议被追问 \xa0 业绩承诺连续三年未完成'}
{'name': '云南龙发制药股份有限公司', 'title': '一份对赌协议引发的风险！融资7200万元，这家新三板公司从计划IPO到面临“易主”'}
{'name': '北京汉邦高科数字技术股份有限公司', 'title': '重组标的业绩承诺未兑现，汉邦高科将回购股份作抵消'}


In [17]:
# 获取公司名称包含 比亚迪 的公司的相关信息
query = """
MATCH (c:Company)-[r:HAPPEN]->(e)
WHERE c.name CONTAINS "比亚迪"
RETURN c.name as name, r.title as title, e.name as event
"""

results = graph.run(query)
for record in results:
    print(dict(record))

{'name': '比亚迪股份有限公司', 'title': '比亚迪上半年新增借款138.7亿 连续三年新增借款超上年净资产20％', 'event': '债务增加'}
{'name': '深圳比亚迪电动汽车投资有限公司', 'title': '比亚迪上半年新增借款138.7亿 连续三年新增借款超上年净资产20％', 'event': '债务增加'}
{'name': '比亚迪股份有限公司', 'title': '方向错了？动力电池市场将再掀起磷酸铁锂热潮', 'event': '产品创新'}
{'name': '深圳比亚迪电动汽车投资有限公司', 'title': '方向错了？动力电池市场将再掀起磷酸铁锂热潮', 'event': '产品创新'}
{'name': '比亚迪股份有限公司', 'title': '车界：新能源车，成也补贴，败也补贴', 'event': '利润下滑'}
{'name': '深圳比亚迪电动汽车投资有限公司', 'title': '车界：新能源车，成也补贴，败也补贴', 'event': '利润下滑'}
{'name': '比亚迪股份有限公司', 'title': '比亚迪(01211.HK)与日野汽车合作研发纯电动商用车', 'event': '业务合作'}
{'name': '深圳比亚迪电动汽车投资有限公司', 'title': '比亚迪(01211.HK)与日野汽车合作研发纯电动商用车', 'event': '业务合作'}
{'name': '比亚迪股份有限公司', 'title': '好事连连', 'event': '引进战略投资'}
{'name': '深圳比亚迪电动汽车投资有限公司', 'title': '好事连连', 'event': '引进战略投资'}
{'name': '比亚迪股份有限公司', 'title': '比亚迪不单单是家车企，这4个副业个个有名，难怪能成领头羊！', 'event': '企业转型'}
{'name': '深圳比亚迪电动汽车投资有限公司', 'title': '比亚迪不单单是家车企，这4个副业个个有名，难怪能成领头羊！', 'event': '企业转型'}


In [ ]:
# 获取投资了 科大讯飞股份有限公司 的投资者
query = """
MATCH (iv:Investor)-[i:INVEST]->(c:Company {name: "科大讯飞股份有限公司"})
RETURN iv.name as investor_name
"""

results = graph.run(query)
for record in results:
    print(dict(record))

{'investor_name': '东方星空投资'}
{'investor_name': '中瑞深圳'}
{'investor_name': '尚珹资本'}
{'investor_name': '聚丰博和'}
{'investor_name': '贵州东方世纪投资管理企业(有限合伙)'}
{'investor_name': '深圳诚成高科'}
{'investor_name': '红十月投资'}
{'investor_name': '廊坊冀财新毅创业引导股权投资基金(有限合伙)'}
{'investor_name': '上海合鲸乐宜投顾'}
{'investor_name': '诚禧投资'}
{'investor_name': '嘉信元德股权投资基金'}
{'investor_name': '方德信'}


In [22]:
# 获取 方德信 投资的公司发生的事件
query = """
MATCH (iv:Investor {name: "方德信"})-[i:INVEST]->(c)-[:HAPPEN]->(e)
RETURN iv.name, c.name, e.name
"""

results = graph.run(query)
for record in results:
    print(dict(record))

{'iv.name': '方德信', 'c.name': '广发证券股份有限公司', 'e.name': '行政整改'}
{'iv.name': '方德信', 'c.name': '广发证券股份有限公司', 'e.name': '自查违规'}
{'iv.name': '方德信', 'c.name': '广发证券股份有限公司', 'e.name': '主体评级下调'}
{'iv.name': '方德信', 'c.name': '广发证券股份有限公司', 'e.name': '董监高—主动离职'}
{'iv.name': '方德信', 'c.name': '科大讯飞股份有限公司', 'e.name': '裁员'}
{'iv.name': '方德信', 'c.name': '科大讯飞股份有限公司', 'e.name': '产品创新'}
{'iv.name': '方德信', 'c.name': '科大讯飞股份有限公司', 'e.name': '股份解禁'}


In [ ]:
# 获取投资了 杭州格像科技有限公司 的投资者还投资了其他哪些公司
query = """
MATCH (other)<-[:INVEST]-(iv:Investor)-[:INVEST]->(c:Company {name :"杭州格像科技有限公司"})
RETURN iv.name, other.name
"""

results = graph.run(query)
for record in results:
    print(dict(record))

{'iv.name': '羽信资本', 'other.name': '西安瑜乐文化科技股份有限公司'}
{'iv.name': '羽信资本', 'other.name': '上海妙克信息科技有限公司'}
{'iv.name': '羽信资本', 'other.name': '深圳市茁壮网络股份有限公司'}
{'iv.name': '广州基金', 'other.name': '苏州朗动网络科技有限公司'}
